<h1 style='color:gray'> SCS 3250-029 Foundations of Data Science</h1>
## Term Project - FIFA World Cup Analysis

In [0]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

import scipy as sp
from scipy.stats import poisson

import warnings
warnings.filterwarnings("ignore")

plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4)

#Set as default behaviour. May be overwritten in code below
pd.options.display.max_rows = 10  


<h1 style='color:green'> Data Collection</h1>
<h3 style='color:black'>Description</h3>

<b>Source</b>: https://www.kaggle.com/abecklas/fifa-world-cup <br>
<b>Context</b>: The FIFA World Cup is a global football competition contested by the senior men's national teams from the 208 Member Associations of FIFA. The competition is held every four years since the inaugural tournament in 1930, with two exceptions. It is the most prestigious and important trophy in the sport of football.
***

In [0]:
world_cup         = pd.read_csv("WorldCups.csv")
world_cup_matches = pd.read_csv("WorldCupMatches.csv")
world_cup_players = pd.read_csv("WorldCupPlayers.csv")

***
<h1 style='color:green'> Data Preparation</h1>
<h3 style='color:black'>Description</h3>

<b>A. Completeness of the Data</b>
1. Treatment of Null Rows / Cells  <br>
2. Treatment of Duplicate Rows<br> 
3. 'Missing Data'  vs 'Not Applicable'

<b>B. Anomalies</b>
1. Wrong Data Type
2. Inconsistency in Names/Spellings and Mojibake
3. Foreign Language Characters

<b> C. Data Tidiness </b>

<b>D. Data Integrity </b>
1. Basic Trends and Coorelation
2. Outlier Identification
***

<h4 style='color:orange'>A. Completeness of the Data</h4>

##### 1/2. Treatment of Null and Duplicate Rows

In [0]:
#World Cups Files
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print("_____Info_____")
    display(world_cup.info())
    print("_____Check for Nulls_____")
    display(world_cup.isnull().sum())
    print("_____Check for Duplicates_____")
    display(world_cup.duplicated().sum()) 

In [0]:
#World Cup Players File
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print("_____Info_____")
    display(world_cup_players.info())
    print("_____Check for Nulls_____")
    display(world_cup_players.isnull().sum())
    print("_____Check for Duplicates_____")
    display(world_cup_players.duplicated().sum())

In [0]:
# In World Cup Players File, drop rows are all-null and drop duplicates, check again.
world_cup_players.dropna(axis=0, how='all', inplace=True)
world_cup_players.drop_duplicates(inplace=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print("_____Check for Nulls_____")
    display(world_cup_players.isnull().sum())
    print("_____Check for Duplicates_____")
    display(world_cup_players.duplicated().sum())

In [0]:
#World Cup Matches File
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print("_____Info_____")
    display(world_cup_matches.info())
    print("_____Check for Nulls_____")
    display(world_cup_matches.isnull().sum())
    print("_____Check for Duplicates_____")
    display(world_cup_matches.duplicated().sum()) 

In [0]:
# In World Cup Matches File, drop rows are all-null and drop duplicates, check again.
world_cup_matches.dropna(axis=0, how='all', inplace=True)
world_cup_matches.drop_duplicates(inplace=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print("_____Check for Nulls_____")
    display(world_cup_matches.isnull().sum())
    print("_____Check for Duplicates_____")
    display(world_cup_matches.duplicated().sum())

Entries are missing the audience data

In [0]:
display(world_cup_matches[world_cup_matches.isnull().any(axis=1)])

##### 3. 'Missing Data' vs 'Not Applicable'? 

<b>For World Cup Players File</b><br>
The following columns contain blank entries that could be better labelled as 'Not Applicable'<br>

<i>Position:</i><br>
C=Captain, GK=Goalkeeper

<i>Event:</i><br>
G=Goal, OG=Own Goal, Y=Yellow Card, R=Red Card, SY = Red Card by second yellow, P=Penalty, MP=Missed Penalty, I = Substitution In, O=Substitute Out

In [0]:
world_cup_players['Event'] = world_cup_players['Event'].fillna('Not Applicable')
world_cup_players['Position'] = world_cup_players['Position'].fillna('Not Applicable')

In [0]:
<b>For World Cup Matches File</b>

For match 300186460, attendance should be: 43,063<br>
Source: https://www.fifa.com/worldcup/matches/round=255951/match=300186460/index.html#lineups#nosticky

In [0]:
display(world_cup_matches.at[823,'Attendance'])
world_cup_matches.at[823,'Attendance'] = 43063
display(world_cup_matches.at[823,'Attendance'])

<h4 style='color:orange'> B. Data Anomalies</h4>
The following anomalies were found as part of data preparation and were corrected.

##### Anomaly # 1 - Wrong Data Type

'Attendance' columns were imported as 'object', but is better suited as an 'integer'. <br>
Columns with year, goals counts, and IDs are also better suited as 'integers'<br>
'Datetime' columns were imported 'object', but is better suited as an 'datetime'. 

In [0]:
# Change the type of Attendance to Integer
world_cup['Attendance'] = world_cup['Attendance'].str.replace('.', '').astype('int64')
world_cup_matches['Attendance'] = world_cup_matches['Attendance'].astype('int64') 

#Columns with year, goals counts, and ids to Integer
world_cup_matches['Year'] = world_cup_matches['Year'].astype('int64') 
world_cup_matches['Home Team Goals'] = world_cup_matches['Home Team Goals'].astype('int64') 
world_cup_matches['Away Team Goals'] = world_cup_matches['Away Team Goals'].astype('int64') 
world_cup_matches['Half-time Home Goals'] = world_cup_matches['Half-time Home Goals'].astype('int64') 
world_cup_matches['Half-time Away Goals'] = world_cup_matches['Half-time Away Goals'].astype('int64') 
world_cup_matches['RoundID'] = world_cup_matches['RoundID'].astype('int64') 
world_cup_matches['MatchID'] = world_cup_matches['MatchID'].astype('int64') 

# Change the type of Datetime to datetime64
world_cup_matches['Datetime'] = world_cup_matches['Datetime'].replace('.', '').astype('datetime64')



In [0]:
# Check the column data types
display(world_cup_matches.info())
display(world_cup_players.info())
display(world_cup.info())

##### Anomaly # 2 - Inconsistency in Names/Spellings and Mojibake
Tables were parsed and reviewed for inconsistency in names and spellings or mojibake (garbled text) e.g. Under the 'Winner' column, there are two different names for Germany: 'Germany FR' and 'Germany'.

World Cup Table

In [0]:
world_cup = world_cup.replace('Germany FR', 'Germany')

In [0]:
# Closer look at Country Names
countries = world_cup_matches.pivot_table(index='Home Team Name', aggfunc='sum')
countries1 = world_cup_matches.pivot_table(index='Away Team Name', aggfunc='sum')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(countries)
    display(countries1)

In [0]:
# Correct Errors
world_cup_matches = world_cup_matches.replace('rn">Bosnia and Herzegovina', 'Bosnia and Herzegovina')
world_cup_matches = world_cup_matches.replace('rn">Republic of Ireland', 'Republic of Ireland')
world_cup_matches = world_cup_matches.replace('rn">Serbia and Montenegro', 'Serbia and Montenegro')
world_cup_matches = world_cup_matches.replace('rn">Trinidad and Tobago', 'Trinidad and Tobago')
world_cup_matches = world_cup_matches.replace('rn">United Arab Emirates', 'United Arab Emirates')

In [0]:
# Correct inconsistencies
world_cup_matches = world_cup_matches.replace('IR Iran', 'Iran')
world_cup_matches = world_cup_matches.replace('Soviet Union', 'Russia')
world_cup_matches = world_cup_matches.replace('Germany FR', 'Germany')
world_cup_matches = world_cup_matches.replace('German DR', 'Germany')

##### Anomaly # 3 - Foreign Language Characters

Foreign language characters we improperly encoded, e.g.: 'Côte d’Ivoire' which appeared as 'C�te d'Ivoire'. These were corrected using a closely related Latin character based on research information.

World Cup:

In [0]:
for col in world_cup.select_dtypes([np.object]).columns[1:]:
    if world_cup.dropna()[col].str.contains('�').any() == True:
        print(col) # Gives the column name(s) with foreign language characters

World Cup Players: <i>Shown for demo, but since this file isn't going to be used, the cleanup of all these characters will be skipped. A sample is shown below.</i>

In [0]:
for col in world_cup_players.select_dtypes([np.object]).columns[1:]:
    if world_cup_players.dropna()[col].str.contains('�').any() == True:
        print(col) # Gives the column name(s) with foreign language characters

In [0]:
print("Coach Name: "+world_cup_players.loc[world_cup_players['Coach Name'].str.contains('�'),'Coach Name'].unique())
print("Player Name: "+world_cup_players.loc[world_cup_players['Player Name'].str.contains('�'),'Player Name'].unique())

In [0]:
#Example of Clean Up
world_cup_players = world_cup_players.replace('PEL� (Edson Arantes do Nascimento)', 'Pelé')
world_cup_players = world_cup_players.replace('Hugo S�NCHEZ', 'Hugo Sánchez')
world_cup_players = world_cup_players.replace('ROM�RIO (Rom�rio de Souza Faria)', 'Romário')

World Cup Matches:

In [0]:
for col in world_cup_matches.select_dtypes([np.object]).columns[1:]:
    if world_cup_matches.dropna()[col].str.contains('�').any() == True:
        print(col) # Gives the column name(s) with foreign language characters

In [0]:
print("Stadium: "+world_cup_matches.loc[world_cup_matches['Stadium'].str.contains('�'), 'Stadium'].unique())
print("City: "+world_cup_matches.loc[world_cup_matches['City']. str.contains('�'), 'City'].unique())
print("Home Team: "+world_cup_matches.loc[world_cup_matches['Home Team Name'].str.contains('�'), 'Home Team Name'].unique())
print("Away Team: "+world_cup_matches.loc[world_cup_matches['Away Team Name'].str.contains('�'), 'Away Team Name'].unique())
print("Referee: "+world_cup_matches.loc[world_cup_matches['Referee'].str.contains('�'), 'Referee'].unique())

In [0]:
world_cup_matches = world_cup_matches.replace("Stade V�lodrome","Stade Vélodrome")
world_cup_matches = world_cup_matches.replace("Maracan� - Est�dio Jornalista M�rio Filho","Maracanã - Estádio Jornalista Mário Filho")
world_cup_matches = world_cup_matches.replace("Nou Camp - Estadio Le�n","Nou Camp")
world_cup_matches = world_cup_matches.replace("Estadio Jos� Mar�a Minella","Estadio José María Minella")
world_cup_matches = world_cup_matches.replace("Estadio Ol�mpico Chateau Carreras","Estadio Olímpico Chateau Carreras")
world_cup_matches = world_cup_matches.replace("Estadio Municipal de Bala�dos","Estadio Municipal de Balaídos")
world_cup_matches = world_cup_matches.replace("Estadio Ol�mpico Universitario","Estadio Olímpico Universitario")
world_cup_matches = world_cup_matches.replace("Malm� ","Malmö ")
world_cup_matches = world_cup_matches.replace("Norrk�Ping ","Norrköping ")
world_cup_matches = world_cup_matches.replace("D�Sseldorf ","Düsseldorf ")
world_cup_matches = world_cup_matches.replace("La Coru�A ","La Coruña ")
world_cup_matches = world_cup_matches.replace("C�te d'Ivoire", "Côte d’Ivoire ")
world_cup_matches = world_cup_matches.replace("St�phane LANNOY (FRA)","Stephane LANNOY (FRA)")
world_cup_matches = world_cup_matches.replace("Oleg�rio BENQUEREN�A (POR)","Olegário BENQUERENÇA (POR)")
world_cup_matches = world_cup_matches.replace("Bj�rn KUIPERS (NED)","Björn KUIPERS (NED)")
world_cup_matches = world_cup_matches.replace("C�neyt �AKIR (TUR)","Cüneyt ÇAKIR (TUR)")

<h4 style='color:orange'> C. Data Tidiness </h4>
The following objectives were adhered to:<br>

•  Each column is a variable<br>
•  Each row is an observation<br>
•  Each type of observational unit forms a table<br>

In [0]:
# Add new column:  'World Cup Number' to world_cup data frame
wc_index = pd.Series(['World Cup 1', 'World Cup 2', 'World Cup 3', 'World Cup 4', 'World Cup 5', 'World Cup 6', 'World Cup 7', 'World Cup 8', 'World Cup 9', 'World Cup 10', 'World Cup 11', 'World Cup 12', 'World Cup 13', 'World Cup 14', 'World Cup 15', 'World Cup 16', 'World Cup 17', 'World Cup 18', 'World Cup 19', 'World Cup 20'])
world_cup['World Cup'] = wc_index 
world_cup = world_cup[['World Cup', 'Year', 'Country', 'Winner', 'Runners-Up', 'Third', 'Fourth', 'GoalsScored', 'QualifiedTeams', 'MatchesPlayed', 'Attendance']]
world_cup.head()

In [0]:
# Reorder World Cup Players data frame
world_cup_players = world_cup_players[['Player Name', 'Team Initials', 'Coach Name', 'Line-up', 'Shirt Number', 'RoundID', 'MatchID', 'Position', 'Event']]
world_cup_players.head()

<h4 style='color:orange'> D. Data Integrity </h4>
• Basic Trends and Correlations<br>
• Outlier Identification

In [0]:
world_cup.plot.scatter(x='MatchesPlayed', y='GoalsScored')

In [0]:
goals = world_cup.pivot_table('GoalsScored', index=['World Cup', 'MatchesPlayed'], aggfunc = 'sum')
goals

In [0]:
world_cup.loc[world_cup['World Cup'] == 'World Cup 5']

NOTE: An outlier was found between 20 and 30 on the x-axis which corresponds with FIFA World Cup # 5 in Switzerland. The entry had an unusually high number of goals  compared to matches played . This outlier was found to be accurate when compared to research.

<h5 style='color:gray'>END OF DATA PREPARTION</h5>
***

<h1 style='color:green'>Data Analysis</h1>

<h2 style='color:black'>Analysis: Attendance</h2>
<h3 style='color:black'>Description</h3>

<b>A. Statistics</b>
1. How many matches have been played in all? Each year?
2. What is the city, stadium, and country with the most matches?

<b>B. Graphs</b>
1. Graph of all attendance over the years in chron order
2. Graph the attendance by day over the years

<b>C. Correlations</b>
1. Determine the three highest attended matches for each cup and make simple observations.
2. Can attendance alone be used to predict a winner?
***

<h4 style='color:orange'>Quick Data Preparation</h4>
#####  Read, Create DataFrames, Sample and Check Metadata

In [0]:
#Create Data Frames
wc_attendance = world_cup_matches[['MatchID','Year','Datetime','Stage','Stadium','City','Home Team Initials','Away Team Initials','Attendance']]
wc_attendance.set_index('MatchID', inplace=True)
wc_attendance.sort_values(by='MatchID', inplace=True)

wc_hosts = pd.read_csv('WorldCupHosts.csv')
wc_hosts.set_index('Year', inplace=True)

#Meta Data and Samples
display(wc_hosts.head())
display(wc_attendance.head())

<h4 style='color:orange'>A. Statistics</h4>

In [0]:
matchcount    = wc_attendance.index.unique().size

#These have unique returns
earliestyear  = wc_attendance.Year.min()
latestyear    = wc_attendance.Year.max()

#These may have ties
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(wc_attendance.groupby('City').City.agg('count').sort_values(ascending=False))
    display(wc_attendance.groupby('Stadium').Stadium.agg('count').sort_values(ascending=False))
    display(wc_hosts.groupby('Host').Host.agg('count').sort_values(ascending=False))

In [0]:
highcountcity = wc_attendance.groupby('City').City.agg('count').max()
highnamecity  = "Mexico City"

highcountstdm = wc_attendance.groupby('Stadium').Stadium.agg('count').max()
highnamestdm  = "Estadio Azteca"

highcountcnty = wc_hosts.groupby('Host').Host.agg('count').max()
highnamecnty  = "Mexico, Italy, Germany, France, and Brazil"


results =\
"There have been a total of "+str(matchcount)+" matches played between "+\
str(earliestyear)+" and "+str(latestyear)+"." + "\nThe country with the most FIFA cups played is "+\
highnamecnty+" ("+str(highcountcnty)+"), \n\tand the city with the most matches is "+(highnamecity)+\
" with a total of "+str(highcountcity)+" matches played there.\nThe stadium, " +(highnamestdm)+\
", has hosted "+str(highcountstdm)+" of these matches, making it the stadium \n\twith the high count of FIFA matches."
print("Results - Fast Facts (as of 2014)\n"+ results)

<h4 style='color:orange'>B. Graphs</h4>

In [0]:
plt.plot(wc_attendance[['Year','Attendance']].groupby('Year').Attendance.agg('sum'),'-')
plt.xlabel('Year')
plt.ylabel('Attendance')
plt.legend()
plt.title('Attendance Over the Years')
plt.show()

In [0]:
val = wc_attendance[['Datetime','Attendance']]
val['Day'] = val.Datetime.dt.weekday_name
val1 = val.groupby('Day').Attendance.agg('sum')
val2 = val.groupby('Day').Attendance.agg('median')
val3 = val.groupby('Day').Attendance.agg('mean')
val4 = val.groupby('Day').Day.agg('count')
days = ['4','0','5','6','3','1','2']  #hardcoded
val5=val1.index
val6 = pd.DataFrame(list(zip(val1,val2,val3,val4,days)), index=val5, 
                    columns=['Sum','Median','Mean','Count','Day_Num'])
val6.sort_values(by="Day_Num",inplace=True)

In [0]:
print("Stadium Attendance By Day")
display(val6)

In [0]:
print("Attenance and Counts")

plt.plot(val6['Day_Num'],val6['Sum'],'-')
plt.ylabel('Total Attendance')
plt.xlabel('Day of the Week')
plt.title('Total Attendance by Day of the Week')
plt.legend()
plt.show()

plt.plot(val6['Day_Num'],val6['Median'],':')
plt.plot(val6['Day_Num'],val6['Mean'],'-')
plt.ylabel('Attendance Mean and Median')
plt.xlabel('Day of the Week')
plt.title('Mean and Median Attendance by Day of the Week')
plt.legend()
plt.show()

plt.plot(val6['Day_Num'],val6['Count'],'-')
plt.ylabel('Number of Matches')
plt.xlabel('Day of the Week')
plt.title('Number of Matches by Day of the Week')

plt.legend()
plt.show()
print("Results - Interesting Findings (as of 2014)\n\
Although historical, Friday is not a popular day to host a match, \
the matches on Fridays are well attended.")

<h4 style='color:orange'>C. Correlations</h4>
#####  1. Observations of the Highest Three Matches

In [0]:
years= wc_attendance.Year.unique()
wc_attendance_top3 = wc_attendance[0:0]
for i in range(0,years.size):
    year_int = int(years[i])
    year_str = str(year_int)
    bool_year = wc_attendance['Year']==year_int
    top3_indices = pd.DataFrame(wc_attendance[bool_year].Attendance.nlargest(3)).index
    top3_records = wc_attendance.loc[[top3_indices[0],top3_indices[1],top3_indices[2]]]
    wc_attendance_top3=wc_attendance_top3.append(top3_records)

In [0]:
print("Total Attendance Number Analysis")
display(wc_attendance.describe(include=[np.number]))
print("Top Three Attendance Number Analysis")
display(wc_attendance_top3.describe(include=[np.number]))
print("Total Attendance Object Analysis")
display(wc_attendance.describe(include=[np.object]))
print("Top Three Attendance Object Analysis")
display(wc_attendance_top3.describe(include=[np.object]))
print("Results - Interesting Findings (as of 2014) \nFinals are \
highly attended matches, and so are matches with BRA or FRG playing. \
Mexico city has high attendance records.")

#####  2. Can attendance in early stages of the cup alone be used to predict a winner? i.e. does the audience have a preference for the games of would be winners? 

Answer: No

In [0]:
def sum_attendance(Year):
    WCYeargame = wc_attendance[wc_attendance['Year']==Year]
    WCYeargame = WCYeargame[WCYeargame['Stage']!= 'Final']
    s1= set(WCYeargame['Away Team Initials'])
    s2= set(WCYeargame['Home Team Initials'])
    s3= s1.union(s2)
    game_sum_attendance = pd.DataFrame(columns=['Team Intials','Total_Sum'])
    game_sum_attendance = game_sum_attendance['Total_Sum'].astype('int64')
    for XXX in s3:
        XXX_A_WCYeargame = WCYeargame[WCYeargame['Away Team Initials'] == XXX]
        XXX_H_WCYeargame = WCYeargame[WCYeargame['Home Team Initials'] == XXX]
        Total_Attendance = XXX_A_WCYeargame.Attendance.sum() + XXX_H_WCYeargame.Attendance.sum()
        df = pd.DataFrame([[XXX, Total_Attendance]], columns=['Team Intials','Total_Sum'])
        game_sum_attendance=game_sum_attendance.append(df, ignore_index=True) 
    return str(game_sum_attendance.loc[game_sum_attendance.Total_Sum.idxmax(),'Team Intials'])

In [0]:
years = [1998, 2010, 2002, 1994, 1986, 1990, 2014]
bool_crit = wc_attendance['Stage']=='Round of 16'
r16_attendance = wc_attendance[bool_crit]

In [0]:
print("Consider seven cups:"+str(years)+"\n\n")
for i in range(0,7):
    year = years[i]
    bool_year = r16_attendance['Year']==year
    r16_foryear_attendance=r16_attendance[bool_year]
    highestHT_index=r16_foryear_attendance.Attendance.idxmax()    
    s= r16_foryear_attendance[['Home Team Initials','Away Team Initials']]
    print("Year: "+str(year) +"\tWinner: "+ wc_hosts.loc[year,'Winner'])
    highest = s.loc[r16_foryear_attendance.Attendance.idxmax()]
    print("> Based on attendance before the final, the winner should be: "+ str(sum_attendance(year)))
    print("> Highest attendence in Round of 16 Stage is Match "+ str(highestHT_index) +"\n"+str(highest)+"\n\n")

<h5 style='color:gray'>END OF ATTENDANCE ANALYSIS</h5>
***

<h2 style='color:black'>Analysis: Matches</h2>
<h3 style='color:black'>Description</h3>

<b>A. Statistics and Graphs</b>
1. Which team has played the most world cup matches?<br><br>

2. Which team could be considered to be the best team?<br>
    a.  Top 3 teams that have rank highly (1, 2, 3) in the the FIFA World Cup Tournament<br>
    b.  Top 3 teams that have rank highly (1, 2, 3) in the the FIFA World Cup Tournament 1998 to 2014 (number of qualifying teams increased to 32 starting from 1998 World Cup) <br>
    c.  Top 10 teams have achieved the most goals overall, and per match<br>

<b>B. Analysis</b>
3.  Which team can best turnaround the game i.e. which teams seem to be losing at half-time, but pull it together in the second half to become the winner of that match?<br>
    a.  How many games can be marked as turnaround game<br>
    b.  Which team get the most times to achieve turnaround<br>


<b>C. Predicting Qualifying Teams</b><br>
Statement on results
***

<h4 style='color:orange'>A. Statistics and Graphs</h4>

##### 1. Team Participation in the World Cup

Which team has attended the highest number of World Cup games?

In [0]:
# Add the Home Team and Away Team as the Teams, then drop the duplicate, get the final particpated teams of every World Cup.
column_update = ['Year', 'Team']
df_home_teams = world_cup_matches[['Year', 'Home Team Name']]
df_home_teams.columns = column_update
df_away_teams = world_cup_matches[['Year', 'Away Team Name']]
df_away_teams.columns = column_update
df_parcipated_teams = pd.concat([df_home_teams,df_away_teams], ignore_index=True)  
df_parcipated_teams.drop_duplicates(subset=None, keep='first', inplace=True)
#df_parcipated_teams.head(20)

In [0]:
df_parcipated_teams_times = df_parcipated_teams['Team'].value_counts()
df_parcipated_teams_times.plot.bar(yticks = pd.Series([0,5,10,15,20]), figsize = (18, 8), grid = True, title="Team Participation in FIFA World Cup Matches")

In [0]:
df_parcipated_teams_times.head(10).plot.bar(yticks = pd.Series([0,5,10,15,20]), figsize = (18, 8), title = 'Top 10 Teams - Team Participation in FIFA World Cup Matches.')

From above picture, Brazil, Italy, Germany, Agentina are the top teams who attended the most times.

##### 2. Which Team Could Be Considered The Best?
<b>2a. Team Placement in the FIFA World Cups - Overall</b>

In [0]:
winner = world_cup["Winner"] # Selecting all the Winners of previous World Cups
runner_up = world_cup["Runners-Up"] # Selecting the Runner Up of previous World Cups
third_place = world_cup["Third"] # Selecting the Third Place Holder of previous World Cups

number_winner = pd.DataFrame(winner.value_counts()) # Counting the Winners and putting them in a dataframe
number_second = pd.DataFrame(runner_up.value_counts()) # Counting the Runner Ups and putting them in a dataframe
number_third = pd.DataFrame(third_place.value_counts()) # Counting the Third Place Holders and putting them in a dataframe

# Join Winner, Runners and Third Place Holders to Top Teams
top_teams = number_winner.join(number_second, how='outer').join(number_third, how='outer') 

top_teams = top_teams.sort_values(by=['Winner', 'Runners-Up', 'Third'], ascending=False) # Sorting them

top_teams.plot(kind="bar", title = 'Top Team FIFA World Cup Placement - 1930 to 2014', fontsize=13, figsize=(15, 7),  width=0.7) # Plotting the graph

<b>2b. Team Placement for World Cups - 1998 to 2014 (number of qualifying teams increased to 32 starting from 1998 World Cup)</b>

In [0]:
world32 = world_cup.loc[world_cup['Year'].isin([1998, 2002, 2006, 2010, 2014])] # Selecting only last 5 World Cups

winner = world32["Winner"] # Selecting all the Winners of previous World Cups
runner_up = world32["Runners-Up"] # Selecting the Runner Up of previous World Cups
third_place = world32["Third"] # Selecting the Third Place Holder of previous World Cups

number_winner = pd.DataFrame(winner.value_counts()) # Counting the Winners and putting them in a dataframe
number_second = pd.DataFrame(runner_up.value_counts()) # Counting the Runner Ups and putting them in a dataframe
number_third = pd.DataFrame(third_place.value_counts()) # Counting the Third Place Holders and putting them in a dataframe

# Top Teams 1st, 2nd, and 3rd Place Holders
top_teams_subset = number_winner.join(number_second, how='outer').join(number_third, how='outer') 
top_teams_subset = top_teams_subset.sort_values(by=['Winner', 'Runners-Up', 'Third'], ascending=False) # Sorting them
top_teams_subset.plot(kind="bar", title = 'Top 3 Teams FIFA World Cup Placement - 1998 to 2014', fontsize=13, figsize=(15, 7),  width=0.7) # Plotting the graph

<b>2c. Which Teams Achieved The Most Goals, Overall and per Match?</b>

In [0]:
column_update = ['Team', 'Goals']
df_home_goals = world_cup_matches[['Home Team Name', 'Home Team Goals']]# Selecting all home team goals
df_home_goals.columns = column_update
df_away_goals = world_cup_matches[['Away Team Name', 'Away Team Goals']]# Selecting all away team goals
df_away_goals.columns = column_update

# Combine the the goals of each country - including both as home and away team
df_goals = pd.concat([df_home_goals,df_away_goals], ignore_index=True) 

In [0]:
s_goals = df_goals.groupby('Team')['Goals'].sum().sort_values(ascending=False) # Grouping and sorting the values

# Select the top 20 countries in terms of goals and sorting them
s_percentage = s_goals/s_goals.sum()
s_percentage.sort_values(ascending=False, inplace=True)
s_percentage.head(20).plot(kind='pie', figsize=(10,10), autopct='%.1f%%', 
                           startangle=173, title='Top 20 - Teams Which Scored the Most Total Goals in FIFA World Cup Matches', label='')

In [0]:
s_score = df_goals.groupby('Team')['Goals'].mean()  # Get the sum of all goals
s_score.sort_values(ascending=False, inplace=True)
s_score
s_score.head(10).plot(kind='bar', figsize=(12,6), title='Top 10 - Teams That Scored the Most Goals per Game in FIFA World Cup Matches')

<h4 style='color:orange'>B. Analysis</h4>
##### 1. Teams That Can Turn Around A Game </h3>
<i>Which teams seem to be losing at half-time, but pull it together in the second half to become the winner of that match?<i/>

In [0]:
# The method to get the winners
def find_win_team(df):
    reverser = []
    for i, row in df.iterrows():
        if row['Half-time Home Goals'] < row['Half-time Away Goals'] and row['Home Team Goals'] > row['Away Team Goals']:
            reverser.append(row['Home Team Name'])
        elif row['Half-time Home Goals'] > row['Half-time Away Goals'] and row['Home Team Goals'] < row['Away Team Goals']:
            reverser.append(row['Away Team Name'])
        else:
            reverser.append('Null')
    return reverser

world_cup_matches['Reverser'] = find_win_team(world_cup_matches)
#df_matches.head()

In [0]:
# There are 38 times, the games can marked as the turnaround game.
reverser = world_cup_matches.groupby('Reverser')['Reverser'].count()
reverser.sort_values(ascending=False, inplace=True)
reverser.drop(labels=['Null'], inplace=True)
print("There are "+str(reverser.sum()) + " games of this type in the history of FIFA.")

In [0]:
reverser.head(5).plot(kind='bar', figsize=(10,6), title='Top 5 Teams That Can Turn Around A Game')

<h4 style='color:orange'>C. Predict Qualifying Teams</h4>
The results of 2a, 2b, and 2c can be used to predict the what teams can qualify for the 2018 FIFA World Cup.

In [0]:
set2a   = set(top_teams.head(3).index.values)
set2b   = set(top_teams_subset.head(3).index.values)
set2ci  = set(s_percentage.head(10).index.values)
set2cii = set(s_score.head(10).index.values)

qual_p = sorted(set2a.union(set2b).union(set2ci).union(set2cii))
print("___BEST-EFFORT PREDICTIVE LIST___")
print("A predictive list of "+str(len(qual_p))+" teams could be:\n")
for item in qual_p : 
    print (item, end='. ') 

In [0]:
print("___WHAT COULD A FULL LIST LOOK LIKE?___")
full_set2a   = set(top_teams.head(5).index.values)
full_set2b   = set(top_teams_subset.head(5).index.values)
full_set2ci  = set(s_percentage.head(25).index.values)
full_set2cii = set(s_score.head(25).index.values)
full_qual_p  = sorted(full_set2a.union(full_set2b).union(full_set2ci).union(full_set2cii))
print("\nA predictive list of "+str(len(full_qual_p))+" teams could be:\n")
for item in full_qual_p : 
    print (item, end='. ')
    
print("\n\n\nNewly Added To List:\n\n"+str(set(full_qual_p).difference(set(qual_p))))

<h5 style='color:gray'>END OF MATCH ANALYSIS</h5>
***

<h2 style='color:black'>Analysis: Predicting the Winner of World Cup </h2>  
<h3 style='color:black'> Description </h3>

<b>A. Statistics: 2018 Participants Quick Facts</b>
1. Qualifying Teams Prediction Performance
2. Teams Participating in the 2018 FIFA World Cup That are First-time Participants
3. Teams Participating in the 2018 FIFA World Cup That Have Never Won A Match
4. Teams Participating in the 2018 FIFA World Cup That Are Not First-time Participants, and Have Never Won a Match
5. Top 20 Match Winners Amongst the Teams Participating in the 2018 FIFA World Cup

<b>B. Prediction: Predicting the Outcome of Matches </b>
1. Set Up the Framework for Prediction Analysis<br>
    a. Calculate the ratio of the goals scored (GS) vs goals against (GA)<br>
    b. Use Poisson distribution to simulate the result of a match<br><br>
    
2. Predict Qualification Teams<br>
    a. Set Up Framework<br>
    b. Calculate Qualification Rate<br><br>
    
3. Determine Outcome of the matches at each elimination stage<br>
    a. Qualification Rate for Group A<br>
    b. Qualification Rate for Group B - Group H<br>
    c. Predict Quarter, Semis, Finals, and Cup Winner<br>

***

<h4 style='color:orange'>A. Statistics: 2018 Partipants Quick Facts </h4>

##### For the 2018 FIFA Cup, there are 32 teams
    Group A：Russia, Egypt, Saudi Arabia,Uruguay
    Group B: Morocco, Spain, Portugal, Iran
    Group C: France, Australia, Peru, Denmark
    Group D: Argentina, Iceland, Croatia, Nigeria
    Group E: Brazil, Costa Rica, Serbia, Switzerland
    Group F: Germany, Mexico, Sweden, Korea Republic
    Group G: Belgium, Panama, Tunisia, England
    Group H: Poland, Senegal, Colombia, Japan

In [0]:
# Create team_list
team_list = ['Russia','Egypt','Saudi Arabia','Uruguay', 
             'Morocco','Spain','Portugal','Iran',
             'France','Australia','Peru','Denmark',
             'Argentina','Iceland', 'Croatia','Nigeria',
             'Brazil','Costa Rica','Serbia','Switzerland',
             'Germany','Mexico','Sweden', 'Korea Republic',
             'Belgium','Panama','Tunisia','England',
             'Poland','Senegal','Colombia','Japan']
df_team_32 = world_cup_matches[(world_cup_matches['Home Team Name'].isin(team_list))&(world_cup_matches['Away Team Name'].isin(team_list))]

##### 1. Qualifying Teams Prediction Performance

In [0]:
print("___HOW DID THE FULL-LIST PREDICTIONS PERFORM?___\n")
realqual_p  = set(team_list)
full_qual_p = set(full_qual_p)
print("Teams that qualified in 2018, that were NOT PREDICTED ["+str(len(realqual_p.difference(full_qual_p)))+"]:  \n" + str(realqual_p.difference(full_qual_p)))
print("\n\nTeams that were predicted to qualify, that DID NOT QUALIFY ["+str(len(full_qual_p.difference(realqual_p)))+"]: \n" + str(full_qual_p.difference(realqual_p)))
print("\n\nTeams that were predicted to qualify and DID QUALIFY ["+str(len(full_qual_p.intersection(realqual_p)))+"]: \n" + str(full_qual_p.intersection(realqual_p)))

##### 2. Teams Participating in the 2018 FIFA World Cup That are First-time Participants

In [0]:
def first_time_to_FIFA(df, team_list):
    first_times = []
    for team in team_list:
        if df_team_32[df_team_32['Home Team Name']==team]['Home Team Name'].count() == 0 and df_team_32[df_team_32['Away Team Name']==team]['Away Team Name'].count() == 0:
               first_times.append(team)        
    return first_times

first_times = first_time_to_FIFA(world_cup_matches, team_list)
print ("Teams entering the 2018 FIFA World Cup Tournament for the first time：")
for item in first_times : 
    print ("\t"+item) 

##### 3/4. Teams Participating in the 2018 FIFA World Cup That Have Never Won A Match & First Time Playing

In [0]:
# List of Winners
def find_win_team(df):
    winners = []
    for i, row in df.iterrows():
        if row['Home Team Goals'] > row['Away Team Goals']:
            winners.append(row['Home Team Name'])
        elif row['Home Team Goals'] < row['Away Team Goals']:
            winners.append(row['Away Team Name'])
        else:
            winners.append('Draw')
    return winners

#Never Won A Match
def never_won_FIFA(df, team_list):
    no_win = []
    for team in team_list:
         if df[df['Winner']==team]['Winner'].count() == 0:
               no_win.append(team)     
    return no_win

In [0]:
df_team_32['Winner'] = find_win_team(df_team_32)
never_won = never_won_FIFA(df_team_32, team_list)

In [0]:
print ("Teams participating in the 2018 who have never won a match:")
for item in never_won :
    print ("\t"+item) 

print ("\nTeams participating in the 2018 who have never won a match, and it's not their first time playing :")
for item in never_won : 
    if item not in first_times:
        print ("\t"+item) 

##### 5. Top 20 Match Winners Amongst the Teams Participating in the 2018 FIFA World Cup

In [0]:
s_32 = df_team_32.groupby('Winner')['Winner'].count()
s_32.sort_values(ascending=False, inplace=True)
s_32.drop(labels=['Draw'], inplace=True)
s_32.sort_values(ascending=True,inplace=True)

s_percentage = s_32/s_32.sum()
s_percentage
s_percentage.tail(20).plot(kind='pie', figsize=(10,10), autopct='%.1f%%', 
                           startangle=173, title='Top 20 Match Winners Amongst the Teams Particpating in the 2018 FIFA World Cup', label='')

<h4 style='color:orange'>B. Prediction: Predicting the Outcomes of Matches</h4>

##### 1. Set Up the Framework for Prediction Analysis
##### 1a. Calculate the ratio of the goals scored (GS) vs goals against (GA)

In [0]:
column_update = ['Team', 'GS', 'GA']
df_score_home_32 = df_team_32[['Home Team Name', 'Home Team Goals', 'Away Team Goals']]
df_score_home_32.columns = column_update
df_score_away_32 = df_team_32[['Away Team Name', 'Away Team Goals', 'Home Team Goals']]
df_score_away_32.columns = column_update
df_32_scores = pd.concat([df_score_home_32,df_score_away_32], ignore_index=True)

In [0]:
# Sum of Goals Scored and Goals Against
s_32_scores = df_32_scores.groupby('Team')['GS', 'GA'].sum() 
s_32_scores['Mean_GS'] = df_32_scores.groupby('Team')['GS'].mean()
s_32_scores['Mean_GA'] = df_32_scores.groupby('Team')['GA'].mean()
s_32_scores

In [0]:
data_new = {'GS':[0,0],
           'GA':[0,0],
           'Mean_GS':[0,0],
           'Mean_GA':[0,0]}
s_32_news = pd.DataFrame(data_new, columns = ['GS', 'GA', 'Mean_GS', 'Mean_GA'], index = ['Iceland','Panama'])
s_32_scores = pd.concat([s_32_scores,s_32_news])
s_32_scores.index.name = 'Team'
s_32_scores

#### 1b. Use Poisson distribution to simulate the result of a match

In [0]:
# Random number for every match
# More times, less random the factors are 
n_sim = 5

def simulate_match(team_A, team_B, knockout=False):
    """simulates one match and returns the goals of the home teams and away teams"""
    # Get the propratily of the goals
    home_scoring_strength = (s_32_scores.loc[team_A, 'Mean_GS'] + s_32_scores.loc[team_B, 'Mean_GA']) / 2
    away_scoring_strength = (s_32_scores.loc[team_A, 'Mean_GA'] + s_32_scores.loc[team_B, 'Mean_GS']) / 2
    # simulate n matches
    fs_A = sp.stats.mode(poisson.rvs(home_scoring_strength, size=n_sim))[0][0]
    fs_B = sp.stats.mode(poisson.rvs(away_scoring_strength, size=n_sim))[0][0]
    # print(team_A, fs_A, team_B, fs_B)
    
    # Knockout Promotion probability 50%：50%
    if knockout:
        if fs_A == fs_B:
            return [team_A, team_B][sp.random.randint(0, 2)]
        elif fs_A > fs_B:
            return team_A
        else:
            return team_B
    return fs_A, fs_B

In [0]:
# Example with Knockout = True , if specified.
simulate_match('Australia', 'France', True)

In [0]:
# Example with Knockout = False (default behaviour) to determine the scores.
simulate_match('Australia', 'France')

##### 2. Simulate X times, at the Group Stage to Predict Qualified Teams
##### 2a. Set Up Framework

In [0]:
## GS（goals scored），GA （goals against），GD（goals difference）。P（points），GP（Games Played）
class Group:
    """Simulate the stage 1"""
    def __init__(self, group_teams, group_name, fixture):
        self.group_teams = group_teams
        self.group_name = group_name
        self.table = pd.DataFrame(0,columns=['Team','GP', 'P', 'GS', 'GA', 'GD'], index=self.group_teams)
        self.fixture = fixture
        self.result = None
        self.qualifiedTeams = []        
    def play(self):
        result = []
        for [team_A, team_B] in self.fixture:
            fs_A, fs_B = simulate_match(team_A, team_B)
            self.table.loc[team_A, 'GP'] += 1
            self.table.loc[team_B, 'GP'] += 1
            self.table.loc[team_A, 'GS'] += fs_A
            self.table.loc[team_B, 'GS'] += fs_B
            self.table.loc[team_A, 'GA'] += fs_B
            self.table.loc[team_B, 'GA'] += fs_A
            if fs_A > fs_B:
                self.table.loc[team_A, 'P'] += 3
            elif fs_A == fs_B:
                self.table.loc[team_A, 'P'] += 1
                self.table.loc[team_B, 'P'] += 1
            elif fs_A < fs_B:
                self.table.loc[team_B, 'P'] += 1
            else:
                raise ValueError('Simulation is error！')
            result.append([team_A, team_B, fs_A, fs_B])
        self.result = pd.DataFrame(result, columns=['Home Team Name', 'Away Team Name', 'Home Team Goals', 'Away Team Goals'])
        self.table['GD'] = self.table['GS'] - self.table['GA']
        self.table['Team'] = self.group_teams                                     
        self.table.sort_values(by=['P', 'GD', 'GS'], 
                               ascending=[False, False, False], inplace=True)    
        self.qualifiedTeams.append(self.table.iat[0,0]) 
        self.qualifiedTeams.append(self.table.iat[1,0]) 

#### 2b. Calculate Qualification Rate

In [0]:
def calculate_qualified_rate(df, times, group_teams, group_name, fixture):
    obj = pd.Series([0,0,0,0], index = group_teams) 
    count = 0
    while (count < times):
       group_test = Group(group_teams,group_name,fixture)   
       group_test.play()
       tmp = group_test.qualifiedTeams[0]
       obj[tmp] =   obj[tmp] + 1
       tmp = group_test.qualifiedTeams[1]
       obj[tmp] =   obj[tmp] + 1
       count += 1
    
    ## Assign the data to dataframe
    df['StageName'] = group_name
    df['SimulateTimes'] = times
    df['QualifiedTeam'] = obj.index.tolist()
    df['WinTimes'] = obj.tolist()
    df['QualifiedRate'] = df['WinTimes']/df['SimulateTimes']
    return obj

In [0]:
def drawQualificationRate(df):       
    df.set_index(df['QualifiedTeam'], inplace= True)
    ymajorFormatter = FormatStrFormatter('%1.1f') 
    yminorLocator   = MultipleLocator(0.1) 
    plt.ylim(0.0, 1.0)        
    df['QualifiedRate'].plot(kind='bar', figsize=(8,4),grid = True)   

In [0]:
data = {'StageName':['GroupA', 'GroupA', 'GroupA', 'GroupA'],
        'SimulateTimes':[0, 0, 0, 0]}
qualificationRate = pd.DataFrame(columns= ['StageName', 'SimulateTimes', 'QualifiedTeam', 'WinTimes', 'QualifiedRate'])
simulat_times = 100

In [0]:
#Simulate Knockout Rounds
## Win（Wins），Lost （Losses），GP（Games Played）
class Knockout:
    """Simulate the Elimination"""
    def __init__(self, group_teams, group_name, fixture, times):
        self.group_teams = group_teams
        self.group_name = group_name
        self.table = pd.DataFrame(0,columns=['GP', 'Win', 'Lost'], index=self.group_teams)
        self.fixture = fixture
        self.simulate_times = times
   
    def play(self):
        result = []
        count = 0
        while(count < self.simulate_times):
            for [team_A, team_B] in self.fixture:
                win_Team = simulate_match(team_A, team_B, True)
                self.table.loc[team_A, 'GP'] += 1
                self.table.loc[team_B, 'GP'] += 1
                if team_A == win_Team:
                    self.table.loc[team_A, 'Win'] += 1
                    self.table.loc[team_B, 'Lost'] += 1
                elif team_B == win_Team:
                    self.table.loc[team_B, 'Win'] += 1
                    self.table.loc[team_A, 'Lost'] += 1
                else:
                    raise ValueError('Simulation is error！')            
            count = count + 1

        self.table.index.name = 'Team'

##### 3a. Qualification Rate for Group A

In [0]:
#Group A：Russia, Egypt, Saudi Arabia, Uruguay
fixture_A = [['Russia', 'Saudi Arabia'], 
             ['Egypt', 'Uruguay'], 
             ['Russia', 'Egypt'], 
             ['Uruguay', 'Saudi Arabia'], 
             ['Saudi Arabia', 'Egypt'], 
             ['Russia', 'Uruguay']]
group_A = ['Russia', 'Egypt', 'Saudi Arabia', 'Uruguay']

group_a = Group(group_A,'Group A',fixture_A)

group_a.play()
group_a.table

In [0]:
qualificationRateA = pd.DataFrame(data, columns= ['StageName', 'SimulateTimes', 'QualifiedTeam', 'WinTimes', 'QualifiedRate'])
obj = calculate_qualified_rate(qualificationRateA,simulat_times, group_A,'Group A',fixture_A)

In [0]:
drawQualificationRate(qualificationRateA)

In [0]:
movingOn = pd.DataFrame()
movingOnA = qualificationRateA.sort_values(by='QualifiedRate',ascending=False).head(2)
movingOn = pd.concat([movingOn,movingOnA], ignore_index=True)

##### 3b. Qualification Rate for Group B -- Group H

In [0]:
#Group B： Morocco, Spain, Portugal, Iran
fixture_B = [['Morocco', 'Portugal'], 
             ['Spain', 'Iran'], 
             ['Morocco', 'Spain'], 
             ['Iran', 'Portugal'], 
             ['Portugal', 'Spain'], 
             ['Morocco', 'Iran']]
group_B = ['Morocco', 'Spain', 'Portugal', 'Iran']

qualificationRateB = pd.DataFrame(data, columns= ['StageName', 'SimulateTimes', 'QualifiedTeam', 'WinTimes', 'QualifiedRate'])
obj = calculate_qualified_rate(qualificationRateB,simulat_times, group_B,'Group B',fixture_B)

movingOnB = qualificationRateB.sort_values(by='QualifiedRate',ascending=False).head(2)
movingOn  = pd.concat([movingOn,movingOnB], ignore_index=True)

In [0]:
#Group C: France, Australia, Peru, Denmark
fixture_C = [['France', 'Australia'], 
             ['Peru', 'Denmark'], 
             ['France', 'Peru'], 
             ['Denmark', 'Australia'], 
             ['Australia', 'Peru'], 
             ['France', 'Denmark']]
group_C = ['France', 'Australia', 'Peru', 'Denmark']

qualificationRateC = pd.DataFrame(data, columns= ['StageName', 'SimulateTimes', 'QualifiedTeam', 'WinTimes', 'QualifiedRate'])
obj = calculate_qualified_rate(qualificationRateC, simulat_times, group_C,'Group C',fixture_C)

movingOnC = qualificationRateC.sort_values(by='QualifiedRate',ascending=False).head(2)
movingOn  = pd.concat([movingOn,movingOnC], ignore_index=True)

In [0]:
#Group D: Argentina, Iceland, Croatia, Nigeria
fixture_D = [['Argentina', 'Iceland'], 
             ['Croatia', 'Nigeria'], 
             ['Argentina', 'Croatia'], 
             ['Nigeria', 'Iceland'], 
             ['Iceland', 'Croatia'], 
             ['Argentina', 'Nigeria']]
group_D = ['Argentina', 'Iceland', 'Croatia', 'Nigeria']

qualificationRateD = pd.DataFrame(data, columns= ['StageName', 'SimulateTimes', 'QualifiedTeam', 'WinTimes', 'QualifiedRate'])
obj = calculate_qualified_rate(qualificationRateD, simulat_times, group_D,'Group D',fixture_D)

movingOnD = qualificationRateD.sort_values(by='QualifiedRate',ascending=False).head(2)
movingOn  = pd.concat([movingOn,movingOnD], ignore_index=True)

In [0]:
#Group E: Brazil, Costa Rica, Serbia, Switzerland
fixture_E = [['Brazil', 'Costa Rica'], 
             ['Serbia', 'Switzerland'], 
             ['Brazil', 'Serbia'], 
             ['Switzerland', 'Costa Rica'], 
             ['Costa Rica', 'Serbia'], 
             ['Brazil', 'Switzerland']]
group_E = ['Brazil', 'Costa Rica', 'Serbia', 'Switzerland']

qualificationRateE = pd.DataFrame(data, columns= ['StageName', 'SimulateTimes', 'QualifiedTeam', 'WinTimes', 'QualifiedRate'])
obj = calculate_qualified_rate(qualificationRateE, simulat_times, group_E,'Group E',fixture_E)

movingOnE = qualificationRateE.sort_values(by='QualifiedRate',ascending=False).head(2)
movingOn  = pd.concat([movingOn,movingOnE], ignore_index=True)

In [0]:
#Group F: Germany, Mexico, Sweden, Korea Republic
fixture_F = [['Germany', 'Mexico'], 
             ['Sweden', 'Korea Republic'], 
             ['Germany', 'Sweden'], 
             ['Korea Republic', 'Mexico'], 
             ['Mexico', 'Sweden'], 
             ['Germany', 'Korea Republic']]
group_F = ['Germany', 'Mexico', 'Sweden', 'Korea Republic']

qualificationRateF = pd.DataFrame(data, columns= ['StageName', 'SimulateTimes', 'QualifiedTeam', 'WinTimes', 'QualifiedRate'])
obj = calculate_qualified_rate(qualificationRateF, simulat_times, group_F,'Group F',fixture_F)

movingOnF = qualificationRateF.sort_values(by='QualifiedRate',ascending=False).head(2)
movingOn  = pd.concat([movingOn,movingOnF], ignore_index=True)

In [0]:
#Group G: Belgium, Panama, Tunisia, England
fixture_G = [['Belgium', 'Panama'], 
             ['Tunisia', 'England'], 
             ['Belgium', 'Tunisia'], 
             ['England', 'Panama'], 
             ['Panama', 'Tunisia'], 
             ['Belgium', 'England']]
group_G = ['Belgium', 'Panama', 'Tunisia', 'England']

qualificationRateG = pd.DataFrame(data, columns= ['StageName', 'SimulateTimes', 'QualifiedTeam', 'WinTimes', 'QualifiedRate'])
obj = calculate_qualified_rate(qualificationRateG, simulat_times, group_G,'Group G',fixture_G)

qualificationRate = pd.concat([qualificationRate,qualificationRateG], ignore_index=True)
movingOnG = qualificationRateG.sort_values(by='QualifiedRate',ascending=False).head(2)
movingOn  = pd.concat([movingOn,movingOnG], ignore_index=True)

In [0]:
#Group H: Poland, Senegal, Colombia, Japan    
fixture_H = [['Poland', 'Senegal'], 
             ['Colombia', 'Japan'], 
             ['Poland', 'Colombia'], 
             ['Japan', 'Senegal'], 
             ['Senegal', 'Colombia'], 
             ['Poland', 'Japan']]
group_H = ['Poland', 'Senegal', 'Colombia', 'Japan']

qualificationRateH = pd.DataFrame(data, columns= ['StageName', 'SimulateTimes', 'QualifiedTeam', 'WinTimes', 'QualifiedRate'])
obj = calculate_qualified_rate(qualificationRateH, simulat_times, group_H,'Group E',fixture_H)

movingOnH = qualificationRateH.sort_values(by='QualifiedRate',ascending=False).head(2)
movingOn  = pd.concat([movingOn,movingOnH], ignore_index=True)

In [0]:
print("Most likely to move on to round of 16:")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(movingOn)

##### 3c. Predict Quarter, Semis, Finals, and Cup Winner

In [0]:
fixture_quarters = [['France', 'Argentina'], 
             ['Uruguay', 'Portugal'], 
             ['Brazil', 'Mexico'], 
             ['Belgium', 'Japan'], 
             ['Spain', 'Russia'], 
             ['Croatia', 'Denmark'],
             ['Sweden', 'Switzerland'],
             ['Colombia', 'England']]
quarter_teams = ['France', 'Argentina', 'Uruguay', 'Portugal',\
                    'Brazil', 'Mexico','Belgium', 'Japan',\
                    'Spain', 'Russia','Croatia', 'Denmark',\
                   'Sweden', 'Switzerland','Colombia', 'England']

games = Knockout(quarter_teams, 'Quarter Finals',fixture_quarters, 100)   
games.play()
print("Quarter Finals")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(games.table)

In [0]:
fixture_semi = [['France', 'Uruguay'], 
             ['Brazil', 'Belgium'], 
             ['Russia', 'Croatia'], 
             ['Sweden', 'England']]
semi_teams = ['France', 'Uruguay', 'Brazil', 'Belgium','Russia', 'Croatia','Sweden', 'England']

games = Knockout(semi_teams, 'Quarter Finals',fixture_semi, 100)   
games.play()
print("Semi Finals")
games.table

In [0]:
fixture_final = [['France', 'Belgium'], 
             ['Croatia', 'England']]
final_teams = ['France', 'Belgium', 'Croatia', 'England']
games = Knockout(final_teams, 'Quarter Finals',fixture_final, 100)   
games.play()
print("Finals")
games.table

In [0]:
fixture_winner= [['France', 'Croatia']]
winner = ['France', 'Croatia']
games = Knockout(winner, 'Quarter Finals',fixture_winner, 100)   
games.play()
print("Cup Winner")
games.table

In [0]:
<h5 style='color:gray'>END OF CUP ANALYSIS</h5>
***

In [0]:
<h1 style='color:gray'> END OF NOTEBOOK </h1>
<h5>Jevonne Peters, Muhammad Kalim, Xiaming Gu</h5>